# In this test file we will use the toffoli gate benchmark to test the functionality of state tomography and our assertion circuit

In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

# Tomography functions
from qiskit.quantum_info import state_fidelity
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#We used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the publically available ibmqx2 backend instead
device = provider.get_backend('ibmqx2')
#if you have access to the twenty qubit machine, use this 20 qubit machine
#device = provider.get_backend('ibmq_boeblingen')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 2
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    allow_q_object: True
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    quantum_volume: 8
    memory: True
    allow_object_storage: True
    backend_name: ibmqx2
    credits_required: True
    conditional: False
    online_date: 2017-01-24T05:00:00+00:00
    url: None
    n_registers: 1
    sample_name: sparrow
    max_shots: 8192
    max_experiments: 75
    open_pulse: False
    description: 5 qubit device
    allow_q_circuit: False

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 34.92678 µs / 36.50602 µs / 0.0 / 0.00058 / 0.00117 / 0.014
    Q1 / 5.23782 GHz / 57.77644 

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

# Test for State Tomography to calculate fidelity

In [5]:
#Toffoli gate
qr = QuantumRegister(3)
circuit = QuantumCircuit(qr)

circuit.ccx(qr[2], qr[1], qr[0])

tomocircuits=state_tomography_circuits(circuit, qr, meas_labels='Pauli', meas_basis='Pauli')
trans_qc = transpile(tomocircuits, device, optimization_level = 3)

In [6]:
jobSim = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi_circ = jobSim.result().get_statevector(circuit)
print(psi_circ)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [7]:
shots = 8192
#execute the job on real device
job = execute(trans_qc, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


In [8]:
tomo_state = StateTomographyFitter(job.result(), tomocircuits)
rho_bell = tomo_state.fit()
F_state = state_fidelity(psi_circ, rho_bell)
print('Fidelity function correct, Toffoli gate fidelity =', F_state)

Fidelity function correct, Toffoli gate fidelity = 0.728026564283783


# Test for assertion circuits and success rate calculation

In [9]:
transQcList = []

#Toffoli gate without assertion:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)

circuit = QuantumCircuit(qr, cr)

circuit.x(qr[1])
circuit.x(qr[2])
circuit.ccx(qr[2], qr[1], qr[0])
circuit.measure(qr, cr)
trans_qc = transpile(circuit, backend=device, optimization_level = 3)
transQcList.append(trans_qc)
print("Toffoli without assertion, size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())

#Toffoli gate with assertion:

qr = QuantumRegister(3)
cr = ClassicalRegister(3)

circuit = QuantumCircuit(qr, cr)

circuit.x(qr[1])
circuit.x(qr[2])
circuit.ccx(qr[2], qr[1], qr[0])
qubitList = [qr[2]]
success = classical_assertion(circuit, qubitList, value = 1)
circuit.measure(qr, cr)

trans_qc = transpile(circuit, backend=device, optimization_level = 3)
transQcList.append(trans_qc)
print("Toffoli with assertion, size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())

Toffoli without assertion, size = 19 depth =  12 width =  8 
 OPs = OrderedDict([('cx', 6), ('u1', 6), ('measure', 3), ('u2', 2), ('u3', 2)]) subcircuits = 3
Toffoli with assertion, size = 31 depth =  17 width =  9 
 OPs = OrderedDict([('cx', 11), ('u3', 8), ('u2', 5), ('measure', 4), ('u1', 3), ('barrier', 1)]) subcircuits = 2


In [10]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of toffoli gate without assertion:

In [11]:
calcSuccessrate(res.get_counts(0), ['111'], 0)

total_count =  8192 success_count =  6115 success_rate =  74.64599609375 %


Success rate of toffoli with assertion:

In [12]:
calcSuccessrate(res.get_counts(1), ['0 111'], 1)

total_count =  7115 success_count =  5494 success_rate =  77.21714687280394 %
